In [1]:
# use print only as a function
from __future__ import print_function

In [37]:
import pandas as pd

# alternative: read file into Pandas using a relative path
path = '../data/yelp.csv'
# Task 1: read the data
ylp = pd.read_csv(path)

In [38]:
# examine the read data
ylp.head(3)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0


In [39]:
# Task 2: Create a new DataFrame that only contains the 5-star and 1-star reviews.
ylp = ylp[(ylp['stars'] == 5) | (ylp['stars'] == 1)]

In [40]:
ylp[(ylp['stars'] == 1)].head(1)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
23,IJ0o6b8bJFAbG6MjGfBebQ,2010-09-05,Dx9sfFU6Zn0GYOckijom-g,1,U can go there n check the car out. If u wanna...,review,zRlQEDYd_HKp0VS3hnAffA,0,1,1


In [41]:
ylp[(ylp['stars'] == 5)].head(1)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0


In [42]:
ylp[(ylp['stars'] == 4)].head(1)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny


In [45]:
# Task 3: create X and y
X = ylp.text
y = ylp.stars

In [49]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)

(3064,)
(1022,)


In [50]:
# Task4: import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [52]:
# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<3064x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 237720 stored elements in Compressed Sparse Row format>

In [54]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1022x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 77006 stored elements in Compressed Sparse Row format>

In [55]:
# Task5: import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [56]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [57]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [58]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.91878669275929548

In [59]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[126,  58],
       [ 25, 813]])

In [60]:
# Task6: calculate the null accuracy
y_test.value_counts()

5    838
1    184
Name: stars, dtype: int64

In [63]:
# most frequent class is 5, calculate null accuracy for class 5
float(y_test.value_counts()[5]) / sum(y_test.value_counts())

0.8199608610567515

In [68]:
# Task 7
# visualize class 1 which were incorrectly predicted as class 5 (False positive)
X_test[y_test < y_pred_class][:5]

2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
Name: text, dtype: object

In [78]:
X_test[y_test < y_pred_class][2175]

'This has to be the worst restaurant in terms of hygiene. Two of my friends had food -poisoning after having dinner here. The food is just unhealthy with tons of oil floating on the top of curries, and I am not sure if any health/hygiene code is followed here. \nThe service is poor and the information on its website is incorrect, the owner does not allow dine-in after 9 or 10 even though it says that the restaurant is open till 11. \n\nOne night I saw the owner cleaning the place without gloves and she was nice enough to give us a to-go parcel without cleaning her hands (great example to the servers!). I had a peek inside the kitchen when the door was ajar, and it definitely looked dirty.\n\nI have been a lot of hole-in-the-wall places around this restaurant, including Haji Baba, the Vietnamese place and others, but neither any of my friends nor I have fallen sick coz of the food. If you need a spicy-food fix, i strongly recommend you do not try this place, lest you want a visit to the

In [66]:
# visualize class 5 which were incorrectly predicted as class 1 (False negative)
X_test[y_test > y_pred_class][:5]

7148    I now consider myself an Arizonian. If you dri...
4963    This is by far my favourite department store, ...
6318    Since I have ranted recently on poor customer ...
380     This is a must try for any Mani Pedi fan. I us...
5565    I`ve had work done by this shop a few times th...
Name: text, dtype: object

In [72]:
X_test[y_test > y_pred_class][7148]

"I now consider myself an Arizonian. If you drive a lot on the 101 or 51 like I do, you'll get your fair share of chips on your windshield. You'll also have to replace a windshield like I had to do just recently. Apparently, chips and cracking windshields  is common in Arizona. In fact, I seem to recall my insurance agent telling me that insurance companies must provide this coverage in Arizona.\n\nI had a chip repaired about a year ago near the very bottom of the windshield. Just recently a small, very fine crack started traveling north on the windshield from the repaired chip (a different vendor repaired the chip). I called these guys over to my house and they said it was too long to fix, so they replaced the whole windshield the next day.\n\nWhat great service, they come out to your residence or place of business to repair or replace your windshield."

In [74]:
# False positive: Even though the review for the restaurant was bad, it was predicted as being good
# False negative: A review that actually should inspire people to go to the restaurant was predicted as being bad

# Both FP & FN reviews are long and therefore most certainly contain a lot of words considered as being 'good' by NB
# as well as words considered to be 'bad'. Like learned in the class a 'positiviness' of each word can be calculate to
# visualize what NB thinks how positive specific words are.


In [79]:
X_train_tokens = vect.get_feature_names()

In [81]:
# number of times each token appears across messages with low rating (bad) [1 star]
bad_token_count = nb.feature_count_[0, :]

In [82]:
# number of times each token appears across messages with high rating (good) [5 star]
good_token_count = nb.feature_count_[1, :]

In [148]:
# create a DataFrame of tokens with their separate good and bad counts
tokens = pd.DataFrame({'token':X_train_tokens, 'bad':bad_token_count, 'good':good_token_count}).set_index('token')

tokens['bad'] = tokens.bad + 1
tokens['good'] = tokens.good + 1
tokens['bad'] = tokens.bad / nb.class_count_[0]
tokens['good'] = tokens.good / nb.class_count_[1]

tokens['positiveness'] = tokens.good / tokens.bad

In [149]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=3)

,bad,good,positiveness
token,,,
amazed,0.00177,0.005202,2.939176
polytechnic,0.00177,0.000800,0.452181
sheared,0.00354,0.000400,0.113045
impersonal,0.00354,0.000400,0.113045
sane,0.00354,0.000400,0.113045


In [152]:
fp_tokens = vect.inverse_transform(X_test_dtm[(y_test < y_pred_class).values][0])[0].tolist()
print(fp_tokens)

[u'you', u'worst', u'without', u'with', u'when', u'website', u'was', u'want', u'wall', u'visit', u'vietnamese', u'very', u'us', u'two', u'try', u'top', u'tons', u'to', u'till', u'though', u'this', u'the', u'that', u'terms', u'sure', u'strongly', u'spicy', u'sick', u'she', u'service', u'servers', u'says', u'saw', u'restaurant', u'recommend', u'poor', u'poisoning', u'places', u'place', u'peek', u'owner', u'others', u'or', u'open', u'one', u'on', u'oil', u'of', u'not', u'nor', u'night', u'nice', u'next', u'neither', u'need', u'my', u'lot', u'looked', u'kitchen', u'just', u'its', u'it', u'is', u'inside', u'information', u'incorrect', u'including', u'in', u'if', u'hygiene', u'hole', u'here', u'her', u'health', u'having', u'have', u'has', u'hands', u'haji', u'had', u'great', u'go', u'gloves', u'give', u'friends', u'food', u'followed', u'floating', u'fix', u'fallen', u'example', u'even', u'enough', u'door', u'does', u'doctor', u'do', u'dirty', u'dinner', u'dine', u'definitely', u'day', u'curr

In [169]:
# I do not exactly know how NB works but for the FP mentioned above the positivness for the positive and negative tokens
# is in the same range if summed up

# I did this to help me see why NB would misclassify the fp. But I do not fully understand the results. 
# What can be observed here is that there are very few positive words (28) but the positivity is very high.
print("Words in FP with positivness > 1 =", len(tokens.loc[fp_tokens][(tokens['positiveness'] > 1)]),
      "| sum(positivness) =", sum(tokens.loc[fp_tokens][(tokens['positiveness'] > 1)].positiveness),
      "\nWords in FP with positivness < 1 =", len(tokens.loc[fp_tokens][(tokens['positiveness'] < 1)]),
      "| sum(positivness) =", sum(tokens.loc[fp_tokens][(tokens['positiveness'] < 1)].positiveness))

Words in FP with positivness > 1 = 28 | sum(positivness) = 43.5417671655 
Words in FP with positivness < 1 = 90 | sum(positivness) = 52.5077953898


/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [171]:
fn_tokens = vect.inverse_transform(X_test_dtm[(y_test > y_pred_class).values][0])[0].tolist()
print(fn_tokens)

[u'your', u'you', u'year', u'windshield', u'whole', u'what', u'was', u'very', u'vendor', u'traveling', u'too', u'to', u'this', u'they', u'these', u'the', u'that', u'telling', u'started', u'so', u'small', u'share', u'service', u'seem', u'said', u'residence', u'replaced', u'replace', u'repaired', u'repair', u'recently', u'recall', u'provide', u'place', u'over', u'out', u'or', u'on', u'of', u'now', u'north', u'next', u'near', u'myself', u'my', u'must', u'me', u'lot', u'long', u'll', u'like', u'just', u'it', u'is', u'insurance', u'in', u'if', u'house', u'have', u'had', u'guys', u'great', u'get', u'from', u'fix', u'fine', u'fair', u'fact', u'drive', u'do', u'different', u'day', u'cracking', u'crack', u'coverage', u'consider', u'companies', u'common', u'come', u'chips', u'chip', u'called', u'business', u'bottom', u'arizona', u'apparently', u'and', u'an', u'also', u'ago', u'agent', u'about', u'51', u'101']


In [172]:
print("Words in FN with positivness > 1 =", len(tokens.loc[fn_tokens][(tokens['positiveness'] > 1)]),
      "| sum(positivness) =", sum(tokens.loc[fn_tokens][(tokens['positiveness'] > 1)].positiveness),
      "\nWords in FN with positivness < 1 =", len(tokens.loc[fn_tokens][(tokens['positiveness'] < 1)]),
      "| sum(positivness) =", sum(tokens.loc[fn_tokens][(tokens['positiveness'] < 1)].positiveness))

Words in FN with positivness > 1 = 18 | sum(positivness) = 27.1616718046 
Words in FN with positivness < 1 = 76 | sum(positivness) = 46.2806494764


/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/tbottesch/.virtualenvs/MLText/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [191]:
# Task 8:
tokens.sort_values("positiveness", ascending=False).head(10)

,bad,good,positiveness
token,,,
fantastic,0.003540,0.077231,21.817727
perfect,0.005310,0.098039,18.464052
yum,0.001770,0.024810,14.017607
favorite,0.012389,0.138055,11.143029
outstanding,0.001770,0.019608,11.078431
brunch,0.001770,0.016807,9.495798
gem,0.001770,0.016006,9.043617
mozzarella,0.001770,0.015606,8.817527
pasty,0.001770,0.015606,8.817527


In [192]:
tokens.sort_values("positiveness", ascending=True).head(10)

,bad,good,positiveness
token,,,
staffperson,0.030088,0.0004,0.013299
refused,0.024779,0.0004,0.016149
disgusting,0.042478,0.0008,0.018841
filthy,0.019469,0.0004,0.020554
unprofessional,0.015929,0.0004,0.025121
unacceptable,0.015929,0.0004,0.025121
acknowledge,0.015929,0.0004,0.025121
ugh,0.030088,0.0008,0.026599
fuse,0.014159,0.0004,0.028261


In [179]:
# Task 9
ylp = pd.read_csv(path)
# Define X and y using the original DataFrame. (y should contain 5 different classes.)
X = ylp.text
y = ylp.stars

In [180]:
# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)

(7500,)
(2500,)


In [181]:
# Create document-term matrices using CountVectorizer.
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test)

In [182]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

In [183]:
metrics.accuracy_score(y_test, y_pred_class)

0.47120000000000001

In [184]:
y_test.value_counts()

4    884
5    832
3    365
2    234
1    185
Name: stars, dtype: int64

In [185]:
# most frequent class is 4, calculate null accuracy for class 4
float(y_test.value_counts()[4]) / sum(y_test.value_counts())

0.3536

In [186]:
# overall accuracy decreased which makes sense since separating 5 classes from each other
# is harder than separating only two. I would expect that classes (1 and 2, 2 and 3 ...) next to each other
# are often confused with each other.

# Regarding the null accuracy. Class 4 and 5 have almost equal size and 'own' together ~70% of the dataset
# If both classes would have perfect predictions we would see an accuracy of 0.7. This is not the case which
# makes one assume that both classes are often confused with each other

In [187]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 55,  14,  24,  65,  27],
       [ 28,  16,  41, 122,  27],
       [  5,   7,  35, 281,  37],
       [  7,   0,  16, 629, 232],
       [  6,   4,   6, 373, 443]])

In [188]:
# like expected class 4 and 5 are often confused
# same for 1 and 2
# it is very strange that so many samples which are actually 2 star ratings are confused with 4 star ratings (122)

In [195]:
from sklearn.metrics import classification_report
target_names = ['1 star ', '2 stars', '3 stars', '4 stars', '5 stars']

print(classification_report(y_test, y_pred_class, target_names=target_names))

             precision    recall  f1-score   support

    1 star        0.54      0.30      0.38       185
    2 stars       0.39      0.07      0.12       234
    3 stars       0.29      0.10      0.14       365
    4 stars       0.43      0.71      0.53       884
    5 stars       0.58      0.53      0.55       832

avg / total       0.46      0.47      0.43      2500



In [ ]:
# The recall score of 2 stars shows that many ratings with != 2 stars are predicted as having 2 stars
# The recall score of 3 stars shows that many ratings with != 3 stars are predicted as having 3 stars

# Many ratings are predicted to be have a 4 star rating while they actually have a different rating